# 求数据集前n个主成分
## 第一主成分

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
X = np.zeros((100, 2))
X[:, 0] = np.random.uniform(0., 100., size=100)
X[:, 1] = 0.75 * X[:, 0] + 3. + np.random.normal(0, 10., size=100)

In [ ]:
plt.scatter(X[:, 0], X[:, 1])
plt.show()

### demean

In [ ]:
def demean(X):
    """特征均值归零化"""
    return X - np.mean(X, axis=0)

In [ ]:
X_demean = demean(X)

In [ ]:
plt.scatter(X_demean[:, 0], X_demean[:, 1])
plt.show()

In [ ]:
np.mean(X_demean[:,0])

In [ ]:
np.mean(X_demean[:,1])

### 梯度上升法

In [ ]:
def f(w, X):
    """目标函数"""
    assert np.alltrue(np.mean(X, axis=0) < 1e-10), "必须先对X进行均值归零化"
    return np.sum(X.dot(w) ** 2) / len(X)

In [ ]:
def df_math(w, X):
    """计算目标函数的梯度"""
    return X.T.dot(X.dot(w)) * 2. / len(X)

In [ ]:
def df_debug(w, X, epsilon=0.0001):
    """梯度粗略值"""
    res = np.empty(len(w))
    for i in range(len(w)):
        w_1 = w.copy()
        w_1[i] += epsilon
        w_2 = w.copy()
        w_2[i] -= epsilon
        res[i] = (f(w_1, X) - f(w_2, X)) / (2 * epsilon)
    return res

In [ ]:
def direction(w):
    """向量单位化"""
    return w / np.linalg.norm(w)

def gradient_ascend(df, X, initial_w, eta, n_iters=1e4, epsilon=1e-8):
    """梯度上升法"""
    
    cur_iter = 0
    w = direction(initial_w)
    
    while cur_iter < n_iters:
        gradient = df(w, X)
        last_w = w
        w = w + eta * gradient
        w = direction(w)  # 注意1: 每次求一个单位方向
        if abs(f(w, X) - f(last_w, X)) < epsilon:
            break
        
        cur_iter += 1
        
    return w

In [ ]:
initial_w = np.random.random(X.shape[1])  # 注意2: 不能用0向量开始
initial_w

In [ ]:
eta = 0.01

In [ ]:
# 注意3: 不能使用StandardScalar标准化数据

In [ ]:
gradient_ascend(df_debug, X_demean, initial_w, eta)

In [ ]:
w = gradient_ascend(df_math, X_demean, initial_w, eta)
w

In [ ]:
plt.scatter(X_demean[:, 0], X_demean[:, 1])
plt.plot([0, w[0]*30], [0, w[1]*30], color='r')
plt.show()

## 第二主成分
### 去掉第一主成分

In [ ]:
def remove_first_component(X, w):
    """去掉第一主成分"""
    return X - X.dot(w).reshape(-1, 1) * w

In [ ]:
X2 = remove_first_component(X, w)

### demean

In [ ]:
X2_demean = demean(X2)

In [ ]:
plt.scatter(X2[:, 0], X2[:, 1])
plt.show()

### 梯度上升法

In [ ]:
w2 = gradient_ascend(df_math, X2_demean, initial_w, eta)
w2

In [ ]:
plt.scatter(X2[:, 0], X2[:, 1])
plt.plot([0, w2[0]*30], [0, w2[1]*30], color='r')
plt.show()

**第一主成分与第二主成分垂直**

In [ ]:
w.dot(w2)

## 前n个主成分

In [ ]:
def first_component(X, initial_w, eta, n_iters=1e4, epsilon=1e-8):
    """利用梯度上升法，求解第一主成分"""
    
    def df(w, X):
        """计算目标函数的梯度"""
        return X.T.dot(X.dot(w)) * 2. / len(X)

    cur_iter = 0
    w = direction(initial_w)

    while cur_iter < n_iters:
        gradient = df(w, X)
        last_w = w
        w = w + eta * gradient
        w = direction(w)  # 注意1: 每次求一个单位方向
        if abs(f(w, X) - f(last_w, X)) < epsilon:
            break

        cur_iter += 1

    return w


def first_n_components(n, X, eta=0.01, n_iters=1e4, epsilon=1e-8):
    """求前n个主成分"""
    
    X_pca = X.copy()
    X_pca = demean(X_pca)
    res = []
    for i in range(n):
        initial_w = np.random.random(X.shape[1])
        w = first_component(X_pca, initial_w, eta, n_iters, epsilon)
        res.append(w)
        X_pca = remove_first_component(X_pca, w)

    return res

In [ ]:
first_n_components(2, X)

# PCA数据降维

In [ ]:
import sys
sys.path.append('playML')
from playML.PCA import PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(X)

In [ ]:
pca.components_

## 降维

In [ ]:
pca = PCA(n_components=1)
pca.fit(X)

In [ ]:
pca.components_

In [ ]:
X_reduction = pca.transform(X)

In [ ]:
X_reduction.shape

## 恢复

In [ ]:
X_restore = pca.inverse_transform(X_reduction)

In [ ]:
X_restore.shape

**降维恢复后的数据**
- pca降维原理: 寻找另外一个坐标系，这个坐标系中的每个轴(主成分)，依次可以表达原来样本(特征)的重要程度。
- pca降维过程: 提取出前k个最重要的主成分，将所有的样本映射到这k个轴上(**会丢失信息**)，获得一个低维的数据。

In [ ]:
plt.scatter(X[:, 0], X[:, 1], color='b', alpha=0.5)
plt.scatter(X_restore[:, 0], X_restore[:, 1], color='r', alpha=0.5)
plt.show()

# scikit-learn中的PCA

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

## 加载数据

In [ ]:
digits = load_digits()
X = digits.data
y = digits.target

## 数据集划分

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666, test_size=0.2)

## 机器学习

### 使用全部数据进行训练

In [ ]:
%%time
estimater = KNeighborsClassifier()
estimater.fit(X_train, y_train)

In [ ]:
score = estimater.score(X_test, y_test)
print("使用全部数据进行训练，准确率为: ", score)

### 使用降维后的数据进行训练

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_train)
X_train_reduction = pca.transform(X_train)
X_test_reduction = pca.transform(X_test)

In [ ]:
%%time
estimater = KNeighborsClassifier()
estimater.fit(X_train_reduction, y_train)

In [ ]:
estimater.score(X_test_reduction, y_test)

#### 讨论：降到多少维合适？
- **explained_variance_ratio_**: 解释每个轴(主成分)可以解释的方差

In [ ]:
pca = PCA(n_components=X_train.shape[1])
pca.fit(X_train)
pca.explained_variance_ratio_

In [ ]:
plt.xlabel("维数")
plt.ylabel("方差")
plt.plot([i for i in range(X_train.shape[1])],
         [np.sum(pca.explained_variance_ratio_[:i+1]) for i in range(len(pca.explained_variance_ratio_))])
plt.show()

- **寻找合适的维度**

In [ ]:
pca = PCA(0.95)  # 传入期望得到的方差，自动计算出要降到的最小维度
pca.fit(X_train)

In [ ]:
pca.n_components_

In [ ]:
X_train_reduction = pca.transform(X_train)
X_test_reduction = pca.transform(X_test)

In [ ]:
%%time
estimater = KNeighborsClassifier()
estimater.fit(X_train_reduction, y_train)

In [ ]:
score = estimater.score(X_test_reduction, y_test)
print(f"降到{pca.n_components_}维进行训练，准确率为: {score}")

- **降到2维进行可视化**

In [ ]:
pca = PCA(n_components=2)
pca.fit(X)
X_reduction = pca.transform(X)

In [ ]:
X_reduction.shape

In [ ]:
for i in range(10):
    plt.scatter(X_reduction[y==i, 0], X_reduction[y==i, 1], alpha=0.8)
plt.show()

# PCA去噪

In [ ]:
from sklearn.datasets import load_digits
from sklearn.decomposition import PCA

In [ ]:
def plot_digits(data):
    "绘制数字"
    fig, axes = plt.subplots(10, 10, figsize=(10, 10),
                             subplot_kw={'xticks': [], 'yticks': []},
                             gridspec_kw=dict(hspace=0.1, wspace=0.1))
    for i, ax in enumerate(axes.flat):
        ax.imshow(data[i].reshape(8, 8),
                  cmap='binary', interpolation='nearest',
                  clim=(0, 16))
    plt.show()
    
def sort_digits(data):
    """对数按行排序"""
    digits = data[y==0, :][:10]
    for num in range(1, 10):
        X_num = data[y==num, :][:10]
        digits = np.vstack([digits, X_num])
    return digits

In [ ]:
# 原始数据集
digits = load_digits()
X = digits.data
y = digits.target
plot_digits(sort_digits(X))

# 加入噪声后的数据集
noisy_digits = X + np.random.normal(0, 4, size=X.shape)
noisy_digits = sort_digits(noisy_digits)
plot_digits(noisy_digits)

# 降噪后的数据集
pca = PCA(0.5)
pca.fit(noisy_digits)
reduction_digits = pca.transform(noisy_digits)
filtered_digits = pca.inverse_transform(reduction_digits)
plot_digits(filtered_digits)